In [ ]:
import pandas as pd
from datetime import timedelta
from joblib import Parallel, delayed
from tqdm.auto import tqdm
import pickle
import datetime
import os
## Project import #
from tfib.evaluations import nDCG_loss
from tfib.evaluations import network_dismantle
from tfib.utils import make_lists_same_length
from tfib.utils import plot_grid_search_heatmap
from tfib.utils import plot_dismantling_graph
from tfib.utils import datetime_to_float
from tfib.utils import generate_splits
from tfib.tfibengines import TFIBEngine

In [ ]:
# SETTINGS #

# Training
DATASET_PATH = "data/COVID19_2023_02_22_DATASET.csv"
TRAIN_SPLIT = 0.8 # Size rateo for training set
VALID_SPLIT = 0.2 # Size rateo for validation set
CREDIBILITY_THR = 39.0 # source > https://www.newsguardtech.com/ratings/rating-process-criteria/
USE_ORIGINAL_RTT = True # Setting to False will use the Jacobson/Karels formula for estimation.
NDCG_AT = 10 # Set the K for NDGC@K. If None the standard nDCG will be computed.

# Execution
PARALLELIZATION_ENABLED = True
N_JOBS = os.cpu_count()

## Data Loading

In [ ]:
dtype = {'tweet_id': str,
         'user_id': str,
         'retweeted_user_id': str,
         'retweeted_status_id': str,
         "retweeeted_user_id": str}

covid_df = pd.read_csv(DATASET_PATH, parse_dates=[1], dtype=dtype)

In [ ]:
covid_df

In [ ]:
data_df = covid_df[covid_df.retweeted_user_id != "AUTHOR"].sort_values(by="created_at").reset_index(drop=True)
data_df

## Datetime Format Conversion 

In [ ]:
data_df = datetime_to_float(data_df, "created_at", time_unit="day")

In [ ]:
data_df

## Train and Test Split

In [ ]:
# Splits
cut_index = int(len(data_df)*TRAIN_SPLIT)
train_set_df = data_df.iloc[:cut_index]
test_set_df = data_df.iloc[cut_index:]

In [ ]:
train_set_df

In [ ]:
test_set_df

## Tuning and Validation Split

- The model will be trained repeatedly on a Tuning set == Training set - Validation set
- The model will be evaluated each time on the same Validation set

In [ ]:
# Split the train set in Tuning and Validation Set
cut_index = int(len(train_set_df)*(1-VALID_SPLIT))
tuning_set_df = train_set_df.iloc[:cut_index]
valid_set_df = train_set_df.iloc[cut_index:]

In [ ]:
tuning_set_df

In [ ]:
valid_set_df

In [ ]:
# Prepare data for hyper-parameter tuning
tuning_set = list(tuning_set_df.itertuples(index=False))

In [ ]:
# Validation ground truth loading
from tfib.evaluations import get_retweet_network, get_optimal_ranking

# Validation retweet network
validation_retweets_network = get_retweet_network(
    valid_set_df[valid_set_df.credibility_score <= CREDIBILITY_THR],
    rt_UserID_col="retweeted_user_id",
    userID_col="user_id",
    rating_col="credibility_score")

# Validation optimal ranking
validation_optimal_ranking = get_optimal_ranking(validation_retweets_network).set_index("node")
validation_optimal_ranking

In [ ]:
# Assign a combined single score to the ranking that keeps the order
validation_optimal_ranking['combined'] = (
    validation_optimal_ranking["outgoing_weight"] +
    validation_optimal_ranking["incoming_weight"] / 
    (validation_optimal_ranking["incoming_weight"].max() + 1))

validation_optimal_ranking

In [ ]:
# Create the ranking in the dict format
validation_optimal_ranking = validation_optimal_ranking.to_dict()["combined"]
list(validation_optimal_ranking.items())[:10]

## Hyperparameters Tuning (Grid Search)

In [ ]:
# Alpha values to explore (importance of the past activity against the last)
alphas = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
# alphas = [0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]

# Beta values to explore (Activity deviation sensitivity)
betas = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

# Gamma values to explore
gammas = [4.0, 3.0, 2.0, 1.0]

# Delta values to explore (Days interval)
max_days = int(tuning_set_df.created_at.iloc[-1])
deltas = generate_splits(max_days, 1.5)

# if standard method is used
if USE_ORIGINAL_RTT:
    # disable beta and gamma
    betas, gammas = [-1], [-1]

# Build hyperparameters configuration set
hparams_set = []
for a in alphas:    
    for b in betas:
        for g in gammas:
            for d in deltas:
                hparams_set.append((a, b, g, d))

# Results storage
evaluated_hparams = []

# Total configurations to test
print(f"Testing {len(hparams_set)} configurations...")

# Kernel function
def test_config(alpha, beta, gamma, delta):

    # create a new model instance with params
    engine = TFIBEngine(
        reshare_key=lambda x: x[0],
        author_key=lambda x: x[2],
        original_post_key=lambda x: x[3],
        original_author_key=lambda x: x[4],
        timestamp_key=lambda x: x[1],
        flag_key=lambda x: x[5],
        credibility_threshold = CREDIBILITY_THR,
        alpha=alpha,
        beta=beta,
        gamma=gamma,
        delta=delta,
        use_original_rtt=USE_ORIGINAL_RTT,
        enable_repost_count_scaling=False)

    # fit the model
    engine.time_fit(tuning_set)

    # get the rank
    ranking = engine.get_rank()
    
    # ranking evaluation
    loss = nDCG_loss(validation_optimal_ranking, ranking, k=NDCG_AT)

    return (alpha, beta, gamma, delta, loss)


# run grid search
if PARALLELIZATION_ENABLED:
    # run in parallel...
    joblib_parallelize = Parallel(n_jobs=N_JOBS, prefer="processes", verbose=10)
    evaluated_hparams = joblib_parallelize(delayed(test_config)(*hparams) for hparams in hparams_set)
else:
    # ...otherwise run serialized
    for hparams in tqdm(hparams_set):
        evaluated_hparams.append(test_config(*hparams))

In [ ]:
# Create a dataframe for evaluated combinations
evaluated_hparams_df = pd.DataFrame(evaluated_hparams,
                                    columns=["Alpha", "Beta", "Gamma", "Delta", "Loss"]).sort_values(by=["Loss", "Delta"])

evaluated_hparams_df.reset_index(inplace=True, drop=True)

evaluated_hparams_df.head(10)

## Grid Search Results Visualization

In [ ]:
# Take the best values
v_alpha, v_beta, v_gamma, v_delta = list(evaluated_hparams_df.iloc[0].values)[:-1]

# Create a new model instance with params
valid_engine = TFIBEngine(
    reshare_key=lambda x: x[0],
    author_key=lambda x: x[2],
    original_post_key=lambda x: x[3],
    original_author_key=lambda x: x[4],
    timestamp_key=lambda x: x[1],
    flag_key=lambda x: x[5],
    credibility_threshold = CREDIBILITY_THR,
    alpha=v_alpha,
    beta=v_beta,
    gamma=v_gamma,
    delta=v_delta,
    use_original_rtt=USE_ORIGINAL_RTT,
    enable_repost_count_scaling=False)

valid_engine.time_fit(tuning_set)
v_ranking = valid_engine.get_rank()

In [ ]:
# Generate the dismantling tracks
v_dismantled_true = network_dismantle(validation_retweets_network, validation_optimal_ranking)
v_dismantled_test = network_dismantle(validation_retweets_network, v_ranking)

# Build the validation dismantling dataframe with results
v_dismantled_df = {}
v_dismantled_df["Validation optimal"] = [x[1] for x in v_dismantled_true]
v_dismantled_df["Validation test"] = [x[1] for x in v_dismantled_test]
make_lists_same_length(v_dismantled_df)
v_dismantled_df = pd.DataFrame.from_dict(v_dismantled_df)

In [ ]:
plot_dismantling_graph(v_dismantled_df, ["darkcyan", "darkblue"], ["*-", "o-"])

### Alpha vs Delta

In [ ]:
# Group by 'Alpha' and 'Delta', then find the row(s) with max 'Loss' in each group
min_loss_rows = evaluated_hparams_df.loc[evaluated_hparams_df.groupby(['Alpha', 'Delta'])['Loss'].idxmin()]

# Pivot the DataFrame
grid_search_df = min_loss_rows.pivot(index='Alpha', columns='Delta', values='Loss')

# Reorder rows and columns in ascending order
grid_search_df.sort_index(axis=0, inplace=True, ascending=False)
grid_search_df.sort_index(axis=1, inplace=True)

In [ ]:
grid_search_df

In [ ]:
plot_grid_search_heatmap(grid_search_df,
                         annot_size=9,
                         palette="magma",
                         color_bar_name="nDCG Loss",
                         color_bar_shrink=0.6,
                         plot_title="Grid Search")

### Setting Hyperparameters for Training Phase

In [ ]:
# Show best and worse hyperparameters found
best_hparams = list(evaluated_hparams_df.iloc[0].values)
worse_hparams = list(evaluated_hparams_df.iloc[-1].values)
print(f"Best hyperparameters: α = {best_hparams[0]}, β = {best_hparams[1]}, γ = {best_hparams[2]}, δ = {best_hparams[3]}")
print(f"Worse hyperparameters: α = {worse_hparams[0]}, β = {worse_hparams[1]}, γ = {worse_hparams[2]}, δ = {worse_hparams[3]}")

In [ ]:
# Set the best and the worse hyperparameters for global usage
BEST_ALPHA, BEST_BETA, BEST_GAMMA, BEST_DELTA = best_hparams[:-1]
WORSE_ALPHA, WORSE_BETA, WORSE_GAMMA, WORSE_DELTA = worse_hparams[:-1]

## Training phase

In [ ]:
# Baseline (FIB-index)
fib_engine_baseline = TFIBEngine(
    reshare_key=lambda x: x[0],
    author_key=lambda x: x[2],
    original_post_key=lambda x: x[3],
    original_author_key=lambda x: x[4],
    timestamp_key=lambda x: x[1],
    flag_key=lambda x: x[5],
    credibility_threshold = CREDIBILITY_THR,
    alpha=BEST_ALPHA,
    beta=BEST_BETA,
    gamma=BEST_GAMMA,
    delta=BEST_DELTA,
    use_original_rtt=USE_ORIGINAL_RTT,
    enable_repost_count_scaling=False)

# Best hyperparameters (TFIB-index)
tfib_engine_best = TFIBEngine(
    reshare_key=lambda x: x[0],
    author_key=lambda x: x[2],
    original_post_key=lambda x: x[3],
    original_author_key=lambda x: x[4],
    timestamp_key=lambda x: x[1],
    flag_key=lambda x: x[5],
    credibility_threshold = CREDIBILITY_THR,
    alpha=BEST_ALPHA,
    beta=BEST_BETA,
    gamma=BEST_GAMMA,
    delta=BEST_DELTA,
    use_original_rtt=USE_ORIGINAL_RTT,
    enable_repost_count_scaling=False)

# Worse hyperparameters (TFIB-index)
tfib_engine_worse = TFIBEngine(
    reshare_key=lambda x: x[0],
    author_key=lambda x: x[2],
    original_post_key=lambda x: x[3],
    original_author_key=lambda x: x[4],
    timestamp_key=lambda x: x[1],
    flag_key=lambda x: x[5],
    credibility_threshold = CREDIBILITY_THR,
    alpha=WORSE_ALPHA,
    beta=WORSE_BETA,
    gamma=WORSE_GAMMA,
    delta=WORSE_DELTA,
    use_original_rtt=USE_ORIGINAL_RTT,
    enable_repost_count_scaling=False)

# Create the data for model training
train_set_tuplelist = list(train_set_df.itertuples(index=False))

# Train both models
fib_engine_baseline.fit(train_set_tuplelist) # Baseline
tfib_engine_best.time_fit(train_set_tuplelist) # Proposed (Best settings)
tfib_engine_worse.time_fit(train_set_tuplelist) # Proposed (Worse settings)

In [ ]:
# Show FIB-i Top-10 (Baseline)
list(fib_engine_baseline.get_rank().items())[:10]

In [ ]:
# Show TFIB-i Top-10 (Proposed best)
list(tfib_engine_best.get_rank().items())[:10]

In [ ]:
# Show TFIB-i Top-10 (Proposed worse)
list(tfib_engine_worse.get_rank().items())[:10]

### Build the Test Ground Truth Ranking

In [ ]:
# Build retweet networks from test set (low credibility retweets only)
test_retweets_network = get_retweet_network(
    test_set_df[test_set_df.credibility_score <= CREDIBILITY_THR],
    rt_UserID_col="retweeted_user_id",
    userID_col="user_id",
    rating_col="credibility_score")

# Raw optimal ranking
test_optimal_ranking = get_optimal_ranking(test_retweets_network).set_index("node")

# Conbine the scores to get a single score that keep the ordering
test_optimal_ranking['combined'] = (
    test_optimal_ranking["outgoing_weight"] +
    test_optimal_ranking["incoming_weight"] / 
    (test_optimal_ranking["incoming_weight"].max() + 1))

# Get the optimal ranking as a dict (ground truth)
test_optimal_ranking = test_optimal_ranking.to_dict()["combined"]

In [ ]:
# Show optimal ranking Top-10
list(test_optimal_ranking.items())[:10]

## Test Loss

In [ ]:
# Evaluate the loss with the test set
FIB_loss_baseline = nDCG_loss(test_optimal_ranking, fib_engine_baseline.get_rank(), k=NDCG_AT)
TFIB_loss_best = nDCG_loss(test_optimal_ranking, tfib_engine_best.get_rank(), k=NDCG_AT)
TFIB_loss_worse = nDCG_loss(test_optimal_ranking, tfib_engine_worse.get_rank(), k=NDCG_AT)

# Show results
print(f"FIB-i Ranking Loss (Baseline): {FIB_loss_baseline}")
print(f"TFIB-i Ranking Loss (Best): {TFIB_loss_best}")
print(f"TFIB-i Ranking Loss (Worse): {TFIB_loss_worse}")

## Results Dataframe

In [ ]:
# Build the results dictionary
test_dismantle_fn = lambda x: network_dismantle(test_retweets_network, x)

results_dict = {
    "FIB-i Baseline": [pair[1] for pair in test_dismantle_fn(fib_engine_baseline.get_rank())],
    f"TFIB-i (α={BEST_ALPHA}, δ={int(BEST_DELTA)})": [pair[1] for pair in test_dismantle_fn(tfib_engine_best.get_rank())],
    f"TFIB-1 (α={WORSE_ALPHA}, δ={int(WORSE_DELTA)})": [pair[1] for pair in test_dismantle_fn(tfib_engine_worse.get_rank())],
    "Optimal": [pair[1] for pair in test_dismantle_fn(test_optimal_ranking)]
}

# Adjust the tracks lenght by replicating the last value
make_lists_same_length(results_dict)

In [ ]:
# Convert to dataframe
results_df = pd.DataFrame(results_dict)
results_df.head(10)

## Hyperparameters Comparison

In [ ]:
# Set up the plot
colors = ["darkorange", "green", "darkred", "darkcyan"]
styles = ["v-", "o-", "x-" , "*-"]

# Plot
plot_dismantling_graph(results_df, colors, styles)

## Saving Results

In [ ]:
# Your dictionary
training_results = {"Alpha": BEST_ALPHA,
                    "Beta": BEST_BETA,
                    "Gamma": BEST_GAMMA,
                    "Delta": BEST_DELTA,
                    "Dataset": DATASET_PATH,
                    "train_split": TRAIN_SPLIT,
                    "valid_split": VALID_SPLIT,
                    "credibility_threshold": CREDIBILITY_THR,
                    "original_RTT": USE_ORIGINAL_RTT,
                    "nDCG@": NDCG_AT,
                    "Ranking": tfib_engine_best.get_rank()}

# Get current timestamp
current_timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")

# Specify the file path
file_path = f"data/training_results_{current_timestamp}.pickle"

# Save the dictionary to disk
with open(file_path, 'wb') as f:
    pickle.dump(training_results, f)

### Results Re-Load Check

In [ ]:
# Load the dictionary from disk
with open(file_path, 'rb') as f:
    loaded_results = pickle.load(f)

import pprint

pprint.pprint(loaded_results, depth=1)